# Ограниченная машина Больцмана (Restricted Boltzmann machine)

## Введение

<b> Ограниченная машина Больцмана (RBM) </b> — это генеративная стохастическая искусственная нейронная сеть, которая определяет распределение вероятности на наборе входных данных.

Как следует из их названия, RBM являются вариантом машин Больцмана с ограничением, что их нейроны должны образовывать двудольный граф: пара узлов, сформированная из нейронов двух групп (обычно называемых «видимыми» и «скрытыми» соответственно), имеет симметричное соединение между собой, и нет никаких связей между узлами внутри группы. Это ограничение позволяет использовать более эффективные алгоритмы обучения, чем те, которые доступны для общего класса машин Больцмана, в частности, алгоритм контрастной дивергенции на основе градиента. Кроме того, особенностью ограниченных машин Больцмана является возможность прохождения обучения без учителя.

## Обозначения и вывод формул

Введем следующие обозначения:
* Состояния нейронов видимого слоя
$$v_1, v_2, ..., v_{N_v}; v_i \in \{0, 1\}, i = \overline{1, N_v}, v \in R^{N_v}$$
* Состояния нейронов скрытого слоя
$$h_1, h_2, ..., h_{N_h}; h_i \in \{0, 1\}, i = \overline{1, N_h}, h \in R^{N_h}$$
* Матрица связей между нейронами видимого и скрытого слоя
$$W \in R^{N_v \times N_h}$$
* Веc между i-ым нейроном видимого слоя и j-ым нейроном скрытого слоя
$$w_{ij}, i = \overline{1, N_v}, j = \overline{1, N_h}$$
* Смещение видимого нейрона
$$a_i, i = \overline{1, N_v}$$
* Смещение скрытого нейрона
$$b_j, j = \overline{1, N_h}$$

<p align="center">
  <img src="https://raw.githubusercontent.com/Frixinglife/RBM/4c4e40b10a6360b641359aef7c5e16dd2ee6190b/images/RBM1.svg" />
</p>

### Энергия и вероятности

* Введем понятие энергии для ограниченной машины Больцмана:
$$E(v, h) = -\sum_{i=1}^{N_v}a_iv_i-\sum_{j=1}^{N_h}b_jh_j-\sum_{i=1}^{N_v}\sum_{j=1}^{N_h}w_{ij}v_ih_j=-a^\top v-b^\top h - v^\top W h$$

* Вероятность всевозможных пар v и h:
$$p(v, h) = \frac{1}{Z}e^{-E(v, h)},$$
где Z — это статсумма следующего вида (суммирование по всевозможным векторам v и h):
$$Z = \sum_{v'}\sum_{h'} e^{-E(v', h')}$$

* Полная вероятность вектора v (суммирование по всевозможным векторам h):
$$p(v) = \frac{1}{Z}\sum_{h'} e^{-E(v, h')}$$

### Вычисление условных вероятностей

* Вероятность того, что при данном v скрытое состояние h<sub>j</sub> = 1:
$$p(h_j = 1|v) = \frac{e^{-E(v,h_j = 1)}}{e^{-E(v,h_j = 1)} + e^{-E(v,h_j = 0)}}=\frac{1}{1 + e^{E(v,h_j = 1)-E(v,h_j = 0)}}=\frac{1}{1 + e^{-b_j-\sum_{i=1}^{N_v}w_{ij}v_i}}=\sigma(b_j+\sum_{i=1}^{N_v}w_{ij}v_i),$$
где
$$\sigma(z)=\frac{1}{1 + e^{-z}}$$
сигмоидальная функция

* Вероятность того, что при данном h видимое состояние v<sub>i</sub> = 1:
$$p(v_i = 1|h) = \frac{e^{-E(v_i = 1,h)}}{e^{-E(v_i = 1,h)} + e^{-E(v_i = 0,h)}}=\frac{1}{1 + e^{E(v_i = 1,h)-E(v_i = 0,h)}}=\frac{1}{1 + e^{-a_i-\sum_{j=1}^{N_h}w_{ij}h_j}}=\sigma(a_i+\sum_{j=1}^{N_h}w_{ij}h_j)$$

* Поскольку при фиксированном v скрытые состояния h<sub>j</sub> независимы друг от друга (как и при фиксированном h видимые состояния v<sub>i</sub>), получаем:
$$p(h|v)=\prod_{j=1}^{N_h}p(h_j|v)$$
$$p(v|h)=\prod_{i=1}^{N_v}p(h|v_i)$$

### Цель обучения

Мы можем наблюдать состояния видимых нейронов, но не можем напрямую увидеть состояния скрытых. Несмотря на это, опираясь на наблюдаемые состояния, можно сделать вероятностный вывод относительно скрытых состояний. После обучения данной модели мы сможем делать вероятностные выводы (при помощи теоремы Байеса) о том, какими являются видимые нейроны, опираясь на скрытые. Таким образом, приходим к следующей <b>цели</b>:

> Необходимо обучить модель таким образом (настроить её параметры), чтобы полученный в результате работы RBM вектор состояния был близок к первоначальному состоянию, из которого он был восстановлен.

Данную цель можно достигнуть путем максимизации вероятности p(v). Кроме того, вместо максимизации вероятности p(v), можно максимизировать натуральный логарифм от нее (это удобнее для получения формул), поскольку он является строго возрастающей функцией.

### Максимизация вероятности p(v)

Найдем производную от натурального логарифма p(v) по весу w<sub>ij</sub>:
$$\frac{\partial \log(p(v))}{\partial w_{ij}} = \frac{1}{p(v)} \frac{\partial p(v)}{\partial w_{ij}} = \frac{1}{p(v)}\frac{\partial}{\partial w_{ij}}(\frac{\sum_{h'} e^{-E(v, h')}}{Z})= \frac{1}{p(v)}\frac{\frac{\partial}{\partial w_{ij}}(\sum_{h'} e^{-E(v, h')})Z - \sum_{h'} e^{-E(v, h')}\frac{\partial Z}{\partial w_{ij}}}{Z^2},$$

где
$$p(v) = \frac{1}{Z}\sum_{h'} e^{-E(v, h')}$$
$$Z = \sum_{v'}\sum_{h'} e^{-E(v', h')}$$
$$E(v, h) = -\sum_{i=1}^{N_v}a_iv_i-\sum_{j=1}^{N_h}b_jh_j-\sum_{i=1}^{N_v}\sum_{j=1}^{N_h}w_{ij}v_ih_j$$

Вычислим производные:
$$\frac{\partial}{\partial w_{ij}}(\sum_{h'} e^{-E(v, h')})=\sum_{h'} v_ih_je^{-E(v, h')}$$
$$\frac{\partial Z}{\partial w_{ij}}=\frac{\partial}{\partial w_{ij}}(\sum_{v'}\sum_{h'} e^{-E(v', h')})=\sum_{v'}\sum_{h'} v_ih_je^{-E(v', h')}$$

После подстановки получаем:
$$\frac{\partial \log(p(v))}{\partial w_{ij}}=\frac{1}{p(v)}\frac{\frac{\partial}{\partial w_{ij}}(\sum_{h'} e^{-E(v, h')})Z - \sum_{h'} e^{-E(v, h')}\frac{\partial Z}{\partial w_{ij}}}{Z^2}=$$
$$=\frac{Z}{\sum_{h'} e^{-E(v, h')}}\frac{Z(\sum_{h'} v_ih_je^{-E(v, h')}) - (\sum_{h'} e^{-E(v, h')})(\sum_{v'}\sum_{h'} v_ih_je^{-E(v', h')})}{Z^2}=$$
$$=\frac{\sum_{h'} v_ih_je^{-E(v, h')}}{\sum_{h'} e^{-E(v, h')}}-\frac{\sum_{v'}\sum_{h'} v_ih_je^{-E(v', h')}}{Z}=$$
$$=(\frac{Z}{\sum_{h'} e^{-E(v, h')}})(\sum_{h'} v_ih_j\frac{e^{-E(v, h')}}{Z})-\sum_{v'}\sum_{h'} v_ih_j\frac{e^{-E(v', h')}}{Z}=$$
$$=\sum_{h'} v_ih_j\frac{p(v,h')}{p(v)}-\sum_{v'}\sum_{h'} v_ih_jp(v',h')=$$
$$=\sum_{h'} v_ih_jp(h'|v)-\sum_{v'}\sum_{h'} v_ih_jp(v',h')$$

Таким образом, получаем:
$$\frac{\partial \log(p(v))}{\partial w_{ij}}=\sum_{h'} v_ih_jp(h'|v)-\sum_{v'}\sum_{h'} v_ih_jp(v',h')=\mathsf{E}_{data}[v_ih_j]-\mathsf{E}_{model}[v_ih_j],$$

где Е - математическое ожидание

Аналогично, можно получить формулы для производных по смещениям:
$$\frac{\partial \log(p(v))}{\partial a_{i}}=\mathsf{E}_{data}[v_i]-\mathsf{E}_{model}[v_i], \frac{\partial \log(p(v))}{\partial b_{j}}=\mathsf{E}_{data}[h_j]-\mathsf{E}_{model}[h_j]$$

### Правила обновления параметров модели

Введем параметр η (learning rate), отвечающий за скорость обучения модели (он влияет на то, насколько сильно изменяются веса при обучении). Тогда получаем следующие правила обновления параметров модели (весов и смещений):
$$\Delta w_{ij}=\eta(\mathsf{E}_{data}[v_ih_j]-\mathsf{E}_{model}[v_ih_j])$$
$$\Delta a_{i}=\eta(\mathsf{E}_{data}[v_i]-\mathsf{E}_{model}[v_i])$$
$$\Delta b_{j}=\eta(\mathsf{E}_{data}[h_j]-\mathsf{E}_{model}[h_j])$$

### Алгоритм обучения Contrastive Divergence (CD-n)

Псевдокод алгоритма является следующим ([ссылка на источник](https://medium.com/machine-learning-researcher/boltzmann-machine-c2ce76d94da5)):

<p align="center">
  <img src="https://raw.githubusercontent.com/Frixinglife/RBM/master/images/CDalgorithm.png" />
</p>

Работа данного алгоритма далее будет рассмотрена на некотором простом примере, но прежде чем к нему перейти, рассмотрим, что понимается под сэмплированием (будет рассмотрена одна из возможных реализаций сэмплирования).

### Сэмплирование вероятностей

Пусть в результате вычисления сигмоидальной функции была получена вероятность 0.6723. Округлим её до двух знаков дробной части и получим число 0.67. То есть:
$$p(h_1 = 1|v) = \sigma(b+W^\top v)_1 = 0.6723 \approx 0.67$$
Теперь можно поступить следующим образом:
> 1. Заведем массив A из 100 элементов, в котором первые 0.67 * 100 = 67 элементов будут единицами, а остальные 100 - 67 = 33 элемента — нулями. 
> 2. Сгенерируем псевдослучайное число следующего вида:
$$\xi \sim U[1, 100],$$
где U[1, 100] — непрерывное равномерное распределение на отрезке [1, 100].
> 3. Будем считать, что (с учетом того, что индексы в массиве начинаются с нуля):
$$h_1 = A[\xi - 1]$$

Можно, например, округлять вероятность до трех знаков дробной части, и заводить массив на 1000 элементов, но это более затратно.

### Пример работы алгоритма

Рассмотрим следующий пример. Пусть имеются данные об оценках трех университетских курсов — математического анализа, линейной алгебры и программирования. Каждому из них студенты поставили оценку 0 или 1, где 1 означает, что предмет студенту нравится, и 0 в противном случае.

| Студент | Математический анализ | Линейная алгебра  | Программирование |
| ------- | :------------------- :| ----------------- | ---------------- |
| Андрей  | 0 | 0 | 1 |
| Павел   | 1 | 0 | 1 |
| Вадим   | 0 | 1 | 0 |

Рассмотрим следующую матрицу:
$$
S = 
\begin{pmatrix}
0&1&0\\
0&0&1\\
1&1&0
\end{pmatrix}
$$

По столбцам в ней записаны значения для нейронов видимого слоя. На первой итерации алгоритма рассмотрим первый столбец матрицы S:
$$
v^{(0)} = 
\begin{pmatrix}
0\\
0\\
1
\end{pmatrix}
$$
Будем считать, что изначально вектора сдвигов являются нулевыми, а матрицу весов заполним случайным образом:
$$
a = 
\begin{pmatrix}
0\\
0\\
0
\end{pmatrix},
b = 
\begin{pmatrix}
0\\
0
\end{pmatrix},
W = 
\begin{pmatrix}
1&-1\\
-2&2\\
1&-1
\end{pmatrix}
$$

Посчитаем вероятности для нейронов скрытого слоя:
$$p(h^{(0)}|v^{(0)})=\sigma(b + W^\top v^{(0)})=\sigma(
\begin{pmatrix}
0\\
0
\end{pmatrix}+
\begin{pmatrix}
1&-2&1\\
-1&2&-1
\end{pmatrix}
\begin{pmatrix}
0\\
0\\
1
\end{pmatrix})=\sigma(
\begin{pmatrix}
1\\
-1
\end{pmatrix})=
\begin{pmatrix}
\sigma(1)\\
\sigma(-1)
\end{pmatrix}=
\begin{pmatrix}
0.7311\\
0.2689
\end{pmatrix}
$$


В соответствии с полученными вероятностями проведем сэмплирование: каждый скрытый нейрон с полученной вероятностью принимает состояние 1. Будем считать, что скрытый нейрон h<sub>1</sub> принимает значение 1 (вероятность больше 0.7), а скрытый нейрон h<sub>2</sub> — значение 0 (вероятность около 0.27). Но в процессе сэмплирования значения могли получиться иными (в данном примере будем считать, что если вероятность больше 0.5, то нейрон принимает значение 1). Таким образом:
$$ h^{(0)} =
\begin{pmatrix}
1\\
0
\end{pmatrix}
$$

Теперь попробуем восстановить исходный вектор, используя h<sup>(0)</sup>:
$$p(v^{(1)}|h^{(0)})=\sigma(a + W h^{(0)})=\sigma(
\begin{pmatrix}
0\\
0\\
0
\end{pmatrix}+
\begin{pmatrix}
1&-1\\
-2&2\\
1&-1
\end{pmatrix}
\begin{pmatrix}
1\\
0\\
\end{pmatrix})=\sigma(
\begin{pmatrix}
1\\
-2\\
1
\end{pmatrix})=
\begin{pmatrix}
\sigma(1)\\
\sigma(-2)\\
\sigma(1)
\end{pmatrix}=
\begin{pmatrix}
0.7311\\
0.1192\\
0.7311
\end{pmatrix}
$$


Предположим, что в процессе сэмплирования видимые нейроны v<sub>1</sub> и v<sub>3</sub> приняли значение 1 (вероятность больше 0.7), а видимый нейрон v<sub>2</sub> — значение 0 (вероятность около 0.12). Получаем:
$$ v^{(1)} =
\begin{pmatrix}
1\\
0\\
1
\end{pmatrix}
$$

Ниже наглядно показана первая итерация алгоритма:
<p align="center">
  <img src="https://raw.githubusercontent.com/Frixinglife/RBM/86172753dd94506a790a37d9f65d37d6c5c127d1/images/CDn1.svg" />
</p>

Будем считать, что мы используем двухшаговый алгоритм контрастной дивергенции (CD-2), поэтому необходимо выполнить еще одну аналогичную итерацию:
$$p(h^{(1)}|v^{(1)})=\sigma(b + W^\top v^{(1)})=\begin{pmatrix}
0.8808\\
0.1192
\end{pmatrix}, h^{(1)} =
\begin{pmatrix}
1\\
0
\end{pmatrix}
$$

$$p(v^{(2)}|h^{(1)})=\sigma(a + W h^{(1)})=
\begin{pmatrix}
0.7311\\
0.1192\\
0.7311
\end{pmatrix}, v^{(2)} =
\begin{pmatrix}
1\\
0\\
1
\end{pmatrix}
$$

<p align="center">
  <img src="https://raw.githubusercontent.com/Frixinglife/RBM/86172753dd94506a790a37d9f65d37d6c5c127d1/images/CDn2.svg" />
</p>

Последний шаг алгоритма завершен, и теперь необходимо обновить веса по следующим формулам (в псевдокоде не учитывался гиперпараметр η, отвечающий за скорость обучения, а точнее за то, насколько сильно меняются веса, но мы его теперь учтем; кроме того, вектора в нем обозначались другими буквами):
$$w_{ij} = w_{ij} + \eta(p(h_j=1|v^{(0)})v_i^{(0)} - p(h_j=1|v^{(k)})v_i^{(k)})$$
$$a_i = a_i + \eta(v_i^{(0)} - v_i^{(k)})$$
$$b_j = b_j + \eta(p(h_j=1|v^{(0)}) - p(h_j=1|v^{(k)})),$$
где k = 2, поскольку используется CD-2, а индексы меняются в следующих диапазонах (N<sub>v</sub> = 3, N<sub>h</sub> = 2):
$$i = \overline{1, N_v}, j = \overline{1, N_h}$$

Для обновления весов нам понадобятся следующие вектора:
$$
v^{(0)} = 
\begin{pmatrix}
0\\
0\\
1
\end{pmatrix}, v^{(2)} =
\begin{pmatrix}
1\\
0\\
1
\end{pmatrix}$$
$$p(h=1|v^{(0)})=\sigma(b + W^\top v^{(0)})=
\begin{pmatrix}
0.7311\\
0.2689
\end{pmatrix}
$$
$$p(h=1|v^{(2)})=\sigma(b + W^\top v^{(2)})=
\begin{pmatrix}
0.8808\\
0.1192
\end{pmatrix}
$$
$$
a = 
\begin{pmatrix}
0\\
0\\
0
\end{pmatrix},
b = 
\begin{pmatrix}
0\\
0
\end{pmatrix},
W = 
\begin{pmatrix}
1&-1\\
-2&2\\
1&-1
\end{pmatrix}
$$
Кроме того, будем считать, что learing rate принимает следующее значение:
$$\eta = 0.5$$

Теперь обновим значения весов:
$$a_1 = 0 + \frac{1}{2}(0 - 1) = -0.5$$
$$a_2 = 0 + \frac{1}{2}(0 - 0) = 0$$
$$a_3 = 0 + \frac{1}{2}(1 - 1) = 0$$

$$b_1 = 0 + \frac{1}{2}(0.7311 - 0.8808) = -0.07485$$
$$b_2 = 0 + \frac{1}{2}(0.2689 - 0.1192) = 0.07485$$

$$w_{11} = 1 + \frac{1}{2}(0.7311 \cdot 0 - 0.8808 \cdot 1) = 0.5596$$
$$w_{12} = -1 + \frac{1}{2}(0.2689 \cdot 0 - 0.1192 \cdot 1) = -1.0596$$
$$w_{21} = -2 + \frac{1}{2}(0.7311 \cdot 0 - 0.8808 \cdot 0) = -2$$
$$w_{22} = 2 + \frac{1}{2}(0.2689 \cdot 0 - 0.1192 \cdot 0) = 2$$
$$w_{31} = 1 + \frac{1}{2}(0.7311 \cdot 1 - 0.8808 \cdot 1) = 1$$
$$w_{32} = -1 + \frac{1}{2}(0.2689 \cdot 1 - 0.1192 \cdot 1) = -1$$
Таким образом, получаем:
$$
a = 
\begin{pmatrix}
-0.5\\
0\\
0
\end{pmatrix},
b = 
\begin{pmatrix}
-0.07485\\
0.07485
\end{pmatrix},
W = 
\begin{pmatrix}
0.5596&-1.0596\\
-2&2\\
0.9251&-0.9251
\end{pmatrix}
$$

<p align="center">
  <img src="https://raw.githubusercontent.com/Frixinglife/RBM/36510a2d58ed6f24b6f737abb48a32b4b09a5f8a/images/CDn3.svg" />
</p>

Для удобства все вычисления выполнялись в среде MatLab. Ограниченная машина Больцмана обучилась на первом векторе входных данных, проведем обучение на остальных:
$$
S = 
\begin{pmatrix}
0&1&0\\
0&0&1\\
1&1&0
\end{pmatrix}
$$

Рассмотрим второй столбец матрицы S:
$$
v^{(0)} = 
\begin{pmatrix}
1\\
0\\
1
\end{pmatrix},a = 
\begin{pmatrix}
-0.5\\
0\\
0
\end{pmatrix},
b = 
\begin{pmatrix}
-0.07485\\
0.07485
\end{pmatrix},
W = 
\begin{pmatrix}
0.5596&-1.0596\\
-2&2\\
0.9251&-0.9251
\end{pmatrix}
$$

$$p(h^{(0)}|v^{(0)})=\sigma(b + W^\top v^{(0)})=\begin{pmatrix}
0.8037\\
0.1289
\end{pmatrix}, h^{(0)} =
\begin{pmatrix}
1\\
0
\end{pmatrix}
$$

$$p(v^{(1)}|h^{(0)})=\sigma(a + W h^{(0)})=
\begin{pmatrix}
0.5149\\
0.1192\\
0.7161
\end{pmatrix}, v^{(1)} =
\begin{pmatrix}
1\\
0\\
1
\end{pmatrix}
$$

$$p(h^{(1)}|v^{(1)})=\sigma(b + W^\top v^{(1)})=\begin{pmatrix}
0.8037\\
0.1289
\end{pmatrix}, h^{(1)} =
\begin{pmatrix}
1\\
0
\end{pmatrix}
$$

$$p(v^{(2)}|h^{(1)})=\sigma(a + W h^{(1)})=
\begin{pmatrix}
0.5149\\
0.1192\\
0.7161
\end{pmatrix}, v^{(2)} =
\begin{pmatrix}
1\\
0\\
1
\end{pmatrix}
$$

Для обновления весов нам понадобятся следующие вектора:
$$p(h=1|v^{(0)})=\sigma(b + W^\top v^{(0)})=
\begin{pmatrix}
0.8037\\
0.1289
\end{pmatrix}
$$
$$p(h=1|v^{(2)})=\sigma(b + W^\top v^{(2)})=
\begin{pmatrix}
0.8037\\
0.1289
\end{pmatrix}
$$
Значения матрицы весов и векторов сдвигов после пересчета не изменились:
$$
a = 
\begin{pmatrix}
-0.5\\
0\\
0
\end{pmatrix},
b = 
\begin{pmatrix}
-0.07485\\
0.07485
\end{pmatrix},
W = 
\begin{pmatrix}
0.5596&-1.0596\\
-2&2\\
0.9251&-0.9251
\end{pmatrix}
$$

Теперь рассмотрим последний столбец матрицы S:
$$
v^{(0)} = 
\begin{pmatrix}
0\\
1\\
0
\end{pmatrix}
$$

$$p(h^{(0)}|v^{(0)})=\sigma(b + W^\top v^{(0)})=\begin{pmatrix}
0.1115\\
0.8884
\end{pmatrix}, h^{(0)} =
\begin{pmatrix}
0\\
1
\end{pmatrix}
$$

$$p(v^{(1)}|h^{(0)})=\sigma(a + W h^{(0)})=
\begin{pmatrix}
0.1737\\
0.8808\\
0.2839
\end{pmatrix}, v^{(1)} =
\begin{pmatrix}
0\\
1\\
0
\end{pmatrix}
$$

$$p(h^{(1)}|v^{(1)})=\sigma(b + W^\top v^{(1)})=\begin{pmatrix}
0.1116\\
0.8884
\end{pmatrix}, h^{(1)} =
\begin{pmatrix}
0\\
1
\end{pmatrix}
$$

$$p(v^{(2)}|h^{(1)})=\sigma(a + W h^{(1)})=
\begin{pmatrix}
0.1737\\
0.8808\\
0.2839
\end{pmatrix}, v^{(2)} =
\begin{pmatrix}
0\\
1\\
0
\end{pmatrix}
$$

Для обновления весов нам понадобятся следующие вектора:
$$p(h=1|v^{(0)})=\sigma(b + W^\top v^{(0)})=
\begin{pmatrix}
0.1115\\
0.8884
\end{pmatrix}
$$
$$p(h=1|v^{(2)})=\sigma(b + W^\top v^{(2)})=
\begin{pmatrix}
0.1115\\
0.8884
\end{pmatrix}
$$
Значения матрицы весов и векторов сдвигов после пересчета снова не изменились:
$$
a = 
\begin{pmatrix}
-0.5\\
0\\
0
\end{pmatrix},
b = 
\begin{pmatrix}
-0.07485\\
0.07485
\end{pmatrix},
W = 
\begin{pmatrix}
0.5596&-1.0596\\
-2&2\\
0.9251&-0.9251
\end{pmatrix}
$$

## Запуск обученной RBM на тестовых данных

Обучение RBM проходило на следующих данных:

| Студент | Математический анализ | Линейная алгебра  | Программирование |
| ------- | :------------------- :| ----------------- | ---------------- |
| Андрей  | 0 | 0 | 1 |
| Павел   | 1 | 0 | 1 |
| Вадим   | 0 | 1 | 0 |

Пусть имеются тестовые данные следующего вида:

| Студент | Математический анализ | Линейная алгебра  | Программирование |
| ------- | :------------------- :| ----------------- | ---------------- |
| Дарья    | 1 | 0 | 1 |
| Анатолий | 0 | 1 | 0 |
| Алексей  | 1 | 1 | 0 |
| Николай  | 0 | 0 | 1 |

Заметим, что почти все строки тестовой выборки совпадают с строками обучающей (кроме имени). Интересно посмотреть, как программа отработает, если ей подать данные, на которых она обучалась. 

Теперь запустим обученную ограниченную машину Больцмана на предложенных тестовых данных (алгоритм остался тем же, только значения матрицы весов и векторов сдвигов больше не обновляются):

$$
Test_{data} = 
\begin{pmatrix}
1&0&1&0\\
0&1&1&0\\
1&0&0&1
\end{pmatrix}, Predict_{data} = 
\begin{pmatrix}
1&0&0&1\\
0&1&1&0\\
1&0&0&1
\end{pmatrix}
$$

Из полученных предсказаний видно, что алгоритм ошибся в двух случаях из четырех. Но это не удивительно, поскольку:
>* Алгоритм обучался на малом количестве входных данных и всего лишь единственный раз (количество эпох равно единице).

>* Можно было бы поэксперементировать и поменять (увеличить) количество шагов метода CD-k и learning rate (попробовать взять его меньше).

>* Сэмплирование происходило не так, как должно быть на самом деле (в данном случае, если вероятность была больше 0.5, то состояние считалось равным 1; вместо этого необходимо использовать алгоритм сэмплирования вероятностей, рассмотренный ранее в одной из глав).

### Скрипт на MATLAB

Все вычисления проводились в MATLAB, и был написан скрипт, который иллюстрирует работу ограниченной машины Больцмана на рассмотренном выше примере. 

Он написан очень подробно, с множеством комментариев, чтобы по нему довольно легко можно было понять, как работает RBM. Его можно посмотреть <b>[здесь](https://github.com/Frixinglife/RBM/blob/master/MATLAB/ExampleRBM.m)</b> (или найти его в репозитории по следующему пути: <b>RBM/MATLAB/ExampleRBM.m</b>).

Кроме того, можно посмотреть на результат его работы для вышеописанного примера (если не хочется запускать сам скрипт). Его можно найти <b>[тут](https://github.com/Frixinglife/RBM/blob/master/MATLAB/ExampleRBM_output.txt)</b> (в репозитории он находится по следующему пути: <b>RBM/MATLAB/ExampleRBM_output.txt</b>).

<b>Спасибо за внимание!</b>

Обратная связь: frixinglife@gmail.com